In [5]:
# Install required libraries
!pip install requests pandas

# Import necessary libraries
import requests
import sqlite3
import pandas as pd

# Task 2: Data Extraction
# API Endpoints
products_url = "https://fakestoreapi.com/products"
carts_url = "https://fakestoreapi.com/carts"
users_url = "https://fakestoreapi.com/users"

# Fetch data from the API
products_data = requests.get(products_url).json()
carts_data = requests.get(carts_url).json()
users_data = requests.get(users_url).json()

# Task 3: Data Storage
# Connect to SQLite database 
conn = sqlite3.connect('fakestore.db')
cursor = conn.cursor()

# Create table schemas
cursor.execute('''CREATE TABLE IF NOT EXISTS products (
                    id INTEGER PRIMARY KEY,
                    title TEXT,
                    price REAL,
                    description TEXT,
                    category TEXT,
                    image TEXT
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS carts (
                    id INTEGER PRIMARY KEY,
                    userId INTEGER,
                    date TEXT,
                    products TEXT
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    email TEXT,
                    username TEXT,
                    password TEXT,
                    name TEXT,
                    address TEXT,
                    phone TEXT
                )''')

# Insert data into the products table
for product in products_data:
    cursor.execute('''INSERT INTO products (id, title, price, description, category, image)
                      VALUES (?, ?, ?, ?, ?, ?)''', 
                      (product['id'], product['title'], product['price'], 
                       product['description'], product['category'], product['image']))

# Insert data into the carts table
for cart in carts_data:
    cursor.execute('''INSERT INTO carts (id, userId, date, products)
                      VALUES (?, ?, ?, ?)''', 
                      (cart['id'], cart['userId'], cart['date'], str(cart['products'])))

# Insert data into the users table
for user in users_data:
    name = f"{user['name']['firstname']} {user['name']['lastname']}"
    address = f"{user['address']['street']}, {user['address']['city']}, {user['address']['zipcode']}"
    cursor.execute('''INSERT INTO users (id, email, username, password, name, address, phone)
                      VALUES (?, ?, ?, ?, ?, ?, ?)''', 
                      (user['id'], user['email'], user['username'], user['password'], 
                       name, address, user['phone']))

# Commit changes and close the connection
conn.commit()
conn.close()

# Load Result into Pandas
# Connect to SQLite database
conn = sqlite3.connect('fakestore.db')

# Load data into Pandas DataFrames
products_df = pd.read_sql_query("SELECT * FROM products", conn)
carts_df = pd.read_sql_query("SELECT * FROM carts", conn)
users_df = pd.read_sql_query("SELECT * FROM users", conn)

# Display the DataFrames
print(products_df.head())
print(carts_df.head())
print(users_df.head())

# Close the database connection
conn.close()


OperationalError: database is locked